In [32]:
#IMPORTING RELEVANT LIBRARIES
import numpy as np
from sklearn import preprocessing

In [33]:
#LOADING THE DATASET
raw_data_csv=np.loadtxt("/content/Audiobooks_data.csv", delimiter=',')

unscaled_input_all = raw_data_csv[:,1:-1]
targets_all = raw_data_csv[:,-1]

In [34]:
#BALANCING THE DATASET
#by visualising we notice that targets contains more 0 than 1's. So, we will count once and then keep 0's count equlas to 1 and delete rest.
num_one_targets=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]

for i in range (targets_all.shape[0]):
  if targets_all[i]==0:
    zero_targets_counter +=1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)
  
unscaled_input_equal_prior = np.delete(unscaled_input_all, indices_to_remove,axis=0)
targets_all_equal_prior= np.delete(targets_all,indices_to_remove,axis=0)

In [35]:
#STANDARDIZING THE INPUTS
scaled_inputs = preprocessing.scale(unscaled_input_equal_prior)

In [42]:
#SHUFFLING THE DATA
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets=targets_all_equal_prior[shuffled_indices]

In [43]:
#SPLITTING INTO TRAIN,TEST,VALIDATION
samples_count= shuffled_inputs.shape[0]

train_sample_count = int(0.8*samples_count)
validation_sample_count = int(0.1*samples_count)
test_sample_count = samples_count - train_sample_count - validation_sample_count

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_inputs = shuffled_inputs[train_sample_count+validation_sample_count:]
test_targets = shuffled_targets[train_sample_count+validation_sample_count:]



#CHECKING THE BALANCING
print(np.sum(train_targets), train_sample_count, np.sum(train_targets)/train_sample_count)
print(np.sum(validation_targets), validation_sample_count, np.sum(validation_targets)/validation_sample_count)
print(np.sum(test_targets), test_sample_count, np.sum(test_targets)/test_sample_count)


1767.0 3579 0.49371332774518023
231.0 447 0.5167785234899329
239.0 448 0.5334821428571429


In [44]:
#SAVING IN SUITABLE FORMAT(.npz)
np.savez('Audiobooks_data_train', inputs=train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

CREATING THE ML ALGORITHM

In [45]:
#IMPORTING RELEVANT LIBRARIES
import numpy as np
import tensorflow as tf

In [46]:
#LOADING THE DATA
npz=np.load('Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(np.float64)
train_targets=npz['targets'].astype(np.int32)

npz=np.load('Audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int32)

npz=np.load('Audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int32)


In [54]:
#ALGORITHM
input_size=10
output_size=2
hidden_layer_size=50
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
    ])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=100
max_epochs=100
early_stopping= tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2
          )


Epoch 1/100
36/36 - 1s - loss: 0.5557 - accuracy: 0.6963 - val_loss: 0.4807 - val_accuracy: 0.7562 - 658ms/epoch - 18ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4547 - accuracy: 0.7695 - val_loss: 0.4236 - val_accuracy: 0.7740 - 125ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4092 - accuracy: 0.7846 - val_loss: 0.3976 - val_accuracy: 0.7808 - 116ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3831 - accuracy: 0.7985 - val_loss: 0.3803 - val_accuracy: 0.7852 - 106ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3709 - accuracy: 0.8036 - val_loss: 0.3711 - val_accuracy: 0.7942 - 110ms/epoch - 3ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3617 - accuracy: 0.7977 - val_loss: 0.3662 - val_accuracy: 0.7808 - 100ms/epoch - 3ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3518 - accuracy: 0.8061 - val_loss: 0.3612 - val_accuracy: 0.7897 - 116ms/epoch - 3ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3480 - accuracy: 0.8083 - val_loss: 0.3478 - val_accuracy: 0.8054 - 101ms/epoch - 3ms/step
Epoch 9

In [57]:
#TESTING
test_loss, test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3709 - accuracy: 0.7946


In [59]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.37. Test accuracy: 79.46%
